In [1]:
import os, time, csv, random
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from collections import Counter
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

from tqdm import tqdm
import optuna
from torch.utils.tensorboard import SummaryWriter



In [2]:
# --- 기본 설정 ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE, torch.cuda.get_device_name(0) if DEVICE.type=="cuda" else "")

Device: cuda NVIDIA GeForce RTX 4070 Laptop GPU


In [3]:
# # --- 데이터셋 경로 및 클래스 정의 ---
# ROOT = Path("/home/wanted-1/PotenupWorkspace/aug-project4/data/Dataset_project4")
ROOT = Path("data/Dataset_project4")

CLASS_NAMES = [
    "metal_can_steel", "metal_can_aluminum", "paper",
    "pet_clear", "pet_colored",
    "plastic_pe", "plastic_pp", "plastic_ps",
    "styrofoam", "vinyl",
    "glass_brown", "glass_green", "glass_clear"
]
CLASS_TO_ID = {c: i for i, c in enumerate(CLASS_NAMES)}
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

def build_df(root: Path):
    paths, labels = [], []
    folder_to_class = {
        "금속캔알루미늄캔": "metal_can_aluminum", "금속캔철캔": "metal_can_steel",
        "비닐": "vinyl", "스티로폼": "styrofoam",
        "유리병_갈색": "glass_brown", "유리병_녹색": "glass_green", "유리병_투명": "glass_clear",
        "종이": "paper", "페트병무색단일": "pet_clear", "페트병유색단일": "pet_colored",
        "플라스틱PE": "plastic_pe", "플라스틱PP": "plastic_pp", "플라스틱PS": "plastic_ps"
    }
    for folder_path in root.iterdir():
        if folder_path.is_dir() and folder_path.name in folder_to_class:
            class_name = folder_to_class[folder_path.name]
            for img_path in folder_path.rglob("*"):
                if img_path.suffix.lower() in IMG_EXTS:
                    paths.append(str(img_path))
                    labels.append(class_name)
    df = pd.DataFrame({"path": paths, "label": labels})
    return df


In [5]:
# --- 데이터셋 분할 및 DataLoader 설정 ---
df = build_df(ROOT)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# NOTE: Optuna 실행 시간을 줄이기 위해 소량의 데이터로 테스트
sample_train_df = (
    train_df.groupby('label', group_keys=False)
    .apply(lambda x: x.sample(n=min(len(x), 100), random_state=42))
    .reset_index(drop=True)
)
sample_test_df = (
    test_df.groupby('label', group_keys=False)
    .apply(lambda x: x.sample(n=min(len(x), 20), random_state=42))
    .reset_index(drop=True)
)
train_df_optuna, test_df_optuna = sample_train_df, sample_test_df
print(f"최적화용 샘플링된 학습 데이터셋 크기: {len(train_df_optuna)}")

최적화용 샘플링된 학습 데이터셋 크기: 1000


C:\Users\user\AppData\Local\Temp\ipykernel_28556\2289011753.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(len(x), 100), random_state=42))
C:\Users\user\AppData\Local\Temp\ipykernel_28556\2289011753.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(len(x), 20), random_state=42))


In [6]:
class CustomDataset(Dataset):
    def __init__(self, df, class_to_id, transform=None):
        self.df = df
        self.class_to_id = class_to_id
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['path']
        label_name = row['label']
        image = Image.open(img_path).convert('RGB')
        label_id = self.class_to_id[label_name]
        if self.transform:
            image = self.transform(image)
        return image, label_id

IMG_SIZE = 512
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset_optuna = CustomDataset(train_df_optuna, CLASS_TO_ID, train_transforms)
test_dataset_optuna = CustomDataset(test_df_optuna, CLASS_TO_ID, test_transforms)

In [7]:
# --- Optuna objective 함수 정의 ---
def objective(trial):
    # 하이퍼파라미터 제안
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    num_epochs = trial.suggest_int('num_epochs', 5, 10)

    # DataLoader 재정의 (batch_size 적용)
    class_counts = train_df_optuna['label'].value_counts()
    num_samples = len(train_df_optuna)
    class_weights = {class_name: num_samples / count for class_name, count in class_counts.items()}
    train_weights = train_df_optuna['label'].apply(lambda x: class_weights[x])
    sampler = WeightedRandomSampler(weights=train_weights.values, num_samples=len(train_weights), replacement=True)
    
    train_loader = DataLoader(train_dataset_optuna, batch_size=batch_size, sampler=sampler, num_workers=0)
    test_loader = DataLoader(test_dataset_optuna, batch_size=batch_size, shuffle=False, num_workers=0)

    # 모델, 손실 함수, 옵티마이저 정의 (하이퍼파라미터 적용)
    model = resnet50(weights=ResNet50_Weights.DEFAULT)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, len(CLASS_NAMES))
    model = model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        # 학습 및 평가 루프 (간소화)
    for epoch in range(num_epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        # 모델 평가 (매 에포크마다 평가할 필요는 없지만, 간단한 예시를 위해 포함)
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        accuracy = accuracy_score(all_labels, all_preds)
    
    return accuracy

In [8]:
# --- Optuna 최적화 실행 ---
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n===========================================")
print("🏆 최적 하이퍼파라미터 및 최고 성능")
print("-------------------------------------------")
print(f"최적의 하이퍼파라미터: {study.best_params}")
print(f"최고 정확도: {study.best_value:.4f}")
print("===========================================")

# --- 최종 모델 학습 (최적의 하이퍼파라미터 사용) ---
print(f"\n최적의 값으로 전체 데이터셋에 대한 최종 모델 학습 시작")

# 최적의 하이퍼파라미터 값 추출
best_lr = study.best_params['learning_rate']
best_bs = study.best_params['batch_size']
best_epochs = study.best_params['num_epochs']

# 전체 데이터셋을 사용하기 위해 train_df, test_df 재정의
df_full = build_df(ROOT)
train_df_full, test_df_full = train_test_split(df_full, test_size=0.2, random_state=42, stratify=df_full['label'])

# DataLoader 재구성 (최적의 batch_size 적용)
train_dataset_full = CustomDataset(train_df_full, CLASS_TO_ID, train_transforms)
test_dataset_full = CustomDataset(test_df_full, CLASS_TO_ID, test_transforms)
class_counts_full = train_df_full['label'].value_counts()
num_samples_full = len(train_df_full)
class_weights_full = {class_name: num_samples_full / count for class_name, count in class_counts_full.items()}
train_weights_full = train_df_full['label'].apply(lambda x: class_weights_full[x])
final_sampler = WeightedRandomSampler(weights=train_weights_full.values, num_samples=len(train_weights_full), replacement=True)

final_train_loader = DataLoader(train_dataset_full, batch_size=best_bs, sampler=final_sampler, num_workers=0)
final_test_loader = DataLoader(test_dataset_full, batch_size=best_bs, shuffle=False, num_workers=0)

# 최종 모델 및 옵티마이저 정의 (최적의 learning_rate 적용)
final_model = resnet50(weights=ResNet50_Weights.DEFAULT)
num_ftrs_final = final_model.fc.in_features
final_model.fc = nn.Linear(num_ftrs_final, len(CLASS_NAMES))
final_model = final_model.to(DEVICE)
final_optimizer = optim.Adam(final_model.parameters(), lr=best_lr)
criterion = nn.CrossEntropyLoss()


[I 2025-08-25 15:16:52,615] A new study created in memory with name: no-name-b349bb69-bb12-4f12-a10a-77e435a049b7


  0%|          | 0/50 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_28556\2879915705.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


[I 2025-08-25 15:30:03,497] Trial 0 finished with value: 0.75 and parameters: {'learning_rate': 0.0001023182681290303, 'batch_size': 16, 'num_epochs': 7}. Best is trial 0 with value: 0.75.


C:\Users\user\AppData\Local\Temp\ipykernel_28556\2879915705.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


[I 2025-08-25 15:36:41,336] Trial 1 finished with value: 0.695 and parameters: {'learning_rate': 0.0003600737092628432, 'batch_size': 8, 'num_epochs': 10}. Best is trial 0 with value: 0.75.


C:\Users\user\AppData\Local\Temp\ipykernel_28556\2879915705.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


[W 2025-08-25 15:38:05,248] Trial 2 failed with parameters: {'learning_rate': 3.8704561107071174e-05, 'batch_size': 8, 'num_epochs': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\project_aug\.venv\lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_28556\2879915705.py", line 33, in objective
    loss.backward()
  File "c:\project_aug\.venv\lib\site-packages\torch\_tensor.py", line 647, in backward
    torch.autograd.backward(
  File "c:\project_aug\.venv\lib\site-packages\torch\autograd\__init__.py", line 354, in backward
    _engine_run_backward(
  File "c:\project_aug\.venv\lib\site-packages\torch\autograd\graph.py", line 829, in _engine_run_backward
    return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt
[W 2025-08-25 15:38:05,289] Trial 2 failed with value No

KeyboardInterrupt: 

In [ ]:
# 최종 학습 루프
for epoch in range(best_epochs):
    final_model.train()
    for images, labels in tqdm(final_train_loader, desc=f"Final Training Epoch {epoch+1}/{best_epochs}"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        final_optimizer.zero_grad()
        outputs = final_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        final_optimizer.step()
    
    # 최종 모델 평가 및 출력 (옵션)
    final_model.eval()
    all_preds_final = []
    all_labels_final = []
    with torch.no_grad():
        for images, labels in final_test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = final_model(images)
            _, preds = torch.max(outputs, 1)
            all_preds_final.extend(preds.cpu().numpy())
            all_labels_final.extend(labels.cpu().numpy())
    
    accuracy_final = accuracy_score(all_labels_final, all_preds_final)
    print(f"\nFinal Epoch {epoch+1}/{best_epochs} Test Accuracy: {accuracy_final:.4f}")


In [ ]:
# 최종 모델 저장
final_model_path = "model/Best_ResNet50_Use_Optuna_model.pth"
if not os.path.exists('model'):
    os.makedirs('model')
torch.save(final_model.state_dict(), final_model_path)
print(f"\n최종 모델이 '{final_model_path}'에 저장되었습니다.")